In [1]:
import glob
import datetime
import numpy as np
import pandas as pd
from library.preprocessing import distance

In [2]:
all_folders=glob.glob("./Trails/*/*/")

test_dates=['2019-06-28','2019-06-30','2019-07-01','2019-07-02','2019-07-03','2019-07-04','2019-07-05']
formatted_test_dates=['-'.join(d.split("-")[::-1]) for d in test_dates] #reversing order

get_date=lambda fname:fname.split("\\")[-2].split("_")[0]
get_format_date=lambda fname: '-'.join(get_date(fname).split("-")[::-1])

#train folders
train_file_pairs=\
[(f+"GPS.csv",
 f+"mu_serv_{}.csv".format(get_format_date(f))) for f in all_folders if get_date(f) not in formatted_test_dates]

#test folders
test_file_pairs=\
[(f+"GPS.csv",
 f+"mu_serv_{}.csv".format(get_format_date(f))) for f in all_folders if get_date(f) in formatted_test_dates]

# Execute for trail

In [47]:
num=25

In [48]:
df_gps=pd.read_csv(test_file_pairs[num][0])
df_mu=pd.read_csv(test_file_pairs[num][1])
print("Opening file:",test_file_pairs[num])

dt=df_gps.time.apply(pd.to_datetime)

Opening file: ('./Trails\\up\\03-07-2019_DATA_08_46_57\\GPS.csv', './Trails\\up\\03-07-2019_DATA_08_46_57\\mu_serv_2019-07-03.csv')


In [49]:
def get_predicted_entry_times(start_time,start_lat,start_long,avg_speed,df_mu):
    for i in range(df_mu.shape[0]):

        next_lat,next_long=df_mu[['lat','long']].iloc[i].values

        travel_time=datetime.timedelta(seconds=round(distance(start_lat,start_long,next_lat,next_long)/avg_speed))

        entry_time=(start_time+travel_time)
        
        #not necessary
        temp_st=pd.to_datetime(str(start_time.date())+" "+df_mu['start_time'].iloc[i])
        late_min=(temp_st.timestamp()-entry_time.timestamp())/60
        print(entry_time,df_mu['start_time'].iloc[i],late_min)
        #to here
        
        exit_time=entry_time+datetime.timedelta(seconds=round(df_mu['estimated_stay_duration'].iloc[i]))

        #change avg speed here
        if i==0:
            #update speed estimate and begining parameters for next pass only once at the begining
            speed_crop=df_gps[(dt>=begin_time) & (dt<=entry_time)]#.speed.replace(0,np.nan).dropna().mean()
            vsc=speed_crop[speed_crop.speed>3].speed
            if len(vsc)==0:
                pass
            else:
                avg_speed=vsc.mean()
    
            start_time_beg=pd.to_datetime(str(start_time.date())+" "+df_mu['start_time'].iloc[i])#exit_time
            start_lat_beg,start_long_beg=next_lat,next_long
            
            

        start_time=exit_time
        start_lat,start_long=next_lat,next_long
    
    return start_time_beg,start_lat_beg,start_long_beg,avg_speed



avg_speed=11 # m/sec
begin_time=pd.to_datetime(df_gps['time'][0]) #static start time
start_time=pd.to_datetime(df_gps['time'][0])
start_lat,start_long=df_gps[['#lat','long']].iloc[0].values


for i in range(df_mu.shape[0]):
    start_time,start_lat,start_long,avg_speed=get_predicted_entry_times(start_time,start_lat,start_long,avg_speed,df_mu.iloc[i:,:])
    print("\n\n\n\n")


2019-07-03 08:47:11 08:47:04 -0.11666666666666667
2019-07-03 08:47:56 08:51:27 3.5166666666666666
2019-07-03 08:48:40 08:51:56 3.2666666666666666
2019-07-03 08:49:32 08:52:44 3.2
2019-07-03 08:50:11 08:55:07 4.933333333333334
2019-07-03 08:50:53 08:55:32 4.65
2019-07-03 08:51:47 08:56:22 4.583333333333333
2019-07-03 08:52:31 08:56:56 4.416666666666667
2019-07-03 08:53:09 08:57:14 4.083333333333333
2019-07-03 08:53:48 08:58:06 4.3
2019-07-03 08:54:03 08:58:24 4.35
2019-07-03 08:54:44 08:58:40 3.933333333333333
2019-07-03 08:55:02 08:59:04 4.033333333333333
2019-07-03 08:55:51 08:59:50 3.9833333333333334
2019-07-03 08:56:38 09:00:40 4.033333333333333
2019-07-03 08:57:34 09:01:38 4.066666666666666
2019-07-03 08:58:13 09:02:06 3.8833333333333333
2019-07-03 08:59:16 09:03:05 3.816666666666667
2019-07-03 09:00:00 09:03:42 3.7
2019-07-03 09:00:54 09:04:27 3.55
2019-07-03 09:01:50 09:05:07 3.283333333333333
2019-07-03 09:02:36 09:05:46 3.1666666666666665
2019-07-03 09:03:19 09:06:12 2.88333333

2019-07-03 08:57:56 08:58:06 0.16666666666666666
2019-07-03 08:58:20 08:58:24 0.06666666666666667
2019-07-03 08:59:14 08:58:40 -0.5666666666666667
2019-07-03 08:59:46 08:59:04 -0.7
2019-07-03 09:00:59 08:59:50 -1.15
2019-07-03 09:02:07 09:00:40 -1.45
2019-07-03 09:03:38 09:01:38 -2.0
2019-07-03 09:04:26 09:02:06 -2.3333333333333335
2019-07-03 09:06:14 09:03:05 -3.15
2019-07-03 09:07:14 09:03:42 -3.533333333333333
2019-07-03 09:08:39 09:04:27 -4.2
2019-07-03 09:10:09 09:05:07 -5.033333333333333
2019-07-03 09:11:15 09:05:46 -5.483333333333333
2019-07-03 09:12:13 09:06:12 -6.016666666666667
2019-07-03 09:12:31 09:06:28 -6.05
2019-07-03 09:13:58 09:07:00 -6.966666666666667
2019-07-03 09:15:06 09:07:25 -7.683333333333334
2019-07-03 09:18:56 09:09:16 -9.666666666666666
2019-07-03 09:20:00 09:10:12 -9.8
2019-07-03 09:20:57 09:12:50 -8.116666666666667
2019-07-03 09:22:30 09:13:59 -8.516666666666667
2019-07-03 09:29:55 09:16:36 -13.316666666666666
2019-07-03 09:31:09 09:18:04 -13.08333333333333

2019-07-03 09:09:42 09:06:28 -3.2333333333333334
2019-07-03 09:11:06 09:07:00 -4.1
2019-07-03 09:12:12 09:07:25 -4.783333333333333
2019-07-03 09:15:50 09:09:16 -6.566666666666666
2019-07-03 09:16:51 09:10:12 -6.65
2019-07-03 09:17:47 09:12:50 -4.95
2019-07-03 09:19:16 09:13:59 -5.283333333333333
2019-07-03 09:26:15 09:16:36 -9.65
2019-07-03 09:27:26 09:18:04 -9.366666666666667
2019-07-03 09:30:21 09:19:47 -10.566666666666666
2019-07-03 09:34:20 09:21:08 -13.2
2019-07-03 09:37:56 09:22:59 -14.95





2019-07-03 09:03:17 09:03:05 -0.2
2019-07-03 09:04:14 09:03:42 -0.5333333333333333
2019-07-03 09:05:34 09:04:27 -1.1166666666666667
2019-07-03 09:06:59 09:05:07 -1.8666666666666667
2019-07-03 09:08:02 09:05:46 -2.2666666666666666
2019-07-03 09:08:58 09:06:12 -2.7666666666666666
2019-07-03 09:09:16 09:06:28 -2.8
2019-07-03 09:10:38 09:07:00 -3.6333333333333333
2019-07-03 09:11:42 09:07:25 -4.283333333333333
2019-07-03 09:15:13 09:09:16 -5.95
2019-07-03 09:16:13 09:10:12 -6.016666666666667
20

In [ ]:
#NICE